In [1]:
import cv2
import sys
import os
#from deepface import DeepFace
import pandas as pd
from tqdm import tqdm

location = str(os.getcwd())

In [5]:
from face_analysis import *

In [3]:
def face_detection(image_path, display = False):
    
    cascPath = "haarcascade_frontalface_default.xml"

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Read the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    i = 0 
    
    file_name = image_path.split("images/")[1]
    file_name = file_name.split(".png")[0]
    
    for (x, y, w, h) in faces:
        file_name_final = file_name + "_" + str(i) + ".jpg"
                
        roi = image[y:y+h, x:x+w]        
        cv2.imwrite(location + "/cropped_faces/" + file_name_final, roi)
    
        i += 1
    
    ### Optional Display Images with Rectangles around Face #####
        if display == True:
            image = cv2.rectangle(image, (x, y), (x + w, y + h), color = (255, 0, 0), thickness = 2)
        
    if display == True:
        cv2.imshow("Faces",image)
        cv2.waitKey(0)
    ################################################################
    
    return(len(faces))



In [7]:
image_df = pd.read_csv("image_info.csv").to_dict('records')

#detect and save faces
for image in tqdm(image_df):
    image_path = location +  "/images/" + image["image_file"]
    try:
        image["detected_faces"] = face_detection(image_path)
    except:
        image["detected_faces"] = 0
        print(image_path)
        
    if image["detected_faces"] > 0:
        face_analysis = face_analysis.analyze(img_path = image_path, actions = ["emotion"], enforce_detection = False)
        image["dominant_emotion"] = [face["dominant_emotion"] for face in face_analysis]
        for emotion in face_analysis[0]["emotion"].keys():
            image[emotion] = [face["emotion"][emotion] for face in face_analysis]
        
#pd.DataFrame(image_df).to_csv("images_processed.csv", index=False)
        
pd.DataFrame(image_df)

  0%|          | 0/1080 [00:00<?, ?it/s]


NameError: name 'face_analysis' is not defined

In [2]:
import pandas as pd

def interpret_str_list(string):
    if type(string) == str:
        new_list = [float(item) for item in string.strip('][').split(', ')]
        return(new_list)
    else:
        return([0])

def condense_emotions(emotion_dict):
    
    anger = max(emotion_dict["angry"]) / -25
    
    sad = max(emotion_dict["sad"]) / -33
    
    fear = max(emotion_dict["fear"]) / -50
    
    disgust = max(emotion_dict["disgust"]) / -100
        
    neutral = 0
    
    happy = max(emotion_dict["happy"]) / 25
    
    surprise = max(emotion_dict["surprise"]) / 25
    
    return(anger + sad + fear + disgust + neutral + happy + surprise)
    

In [3]:
image_df = pd.read_csv("images_processed.csv").to_dict('records')

for image in image_df:
    if image["detected_faces"] > 0:
        emotion_dict = {}
        for emotion in ["angry", "sad", "disgust", "fear", "happy", "surprise", "neutral"]:
            emotion_dict[emotion] = interpret_str_list(image[emotion])        
        
        image["emotion_value"] = condense_emotions(emotion_dict)/len(emotion_dict["angry"])
        
pd.DataFrame(image_df).to_csv("images_processed_clean.csv", index=False)
pd.DataFrame(image_df)

,date,location,url,image_file,detected_faces,dominant_emotion,angry,disgust,fear,happy,sad,surprise,neutral,emotion_value
0,12/11/2011,IUB,https://www.idsnews.com/article/2011/12/iusa-r...,IUB_12112011_1.png,1,['neutral'],[10.125825837156077],[0.00012061833257604476],[5.82172987500073],[0.007551853750704527],[39.85177995541335],[0.17242825397214245],[44.02055811636947],-1.721899
1,11/7/2011,IUB,https://www.idsnews.com/article/2011/11/not-th...,IUB_11072011_3.png,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/27/2011,IUB,https://www.idsnews.com/article/2011/10/not-an...,IUB_10272011_7.png,1,['happy'],[2.9219497187049427e-05],[2.7978699585173844e-10],[6.690121095783776],[90.71392957651196],[2.5657210165691984],[8.291823278176871e-08],[0.030193511682920855],3.417004
3,10/5/2011,IUB,https://www.idsnews.com/article/2011/10/stayin...,IUB_10052011_12.png,1,['sad'],[0.9576824493706226],[0.00013423082236840855],[7.246794551610947],[0.00875094992807135],[88.01890015602112],[0.0005184358542464906],[3.76722551882267],-2.850113
4,10/5/2011,IUB,https://www.idsnews.com/article/2011/10/how-no...,IUB_10052011_13.png,1,['sad'],[1.7567207221222354],[0.0002922457216778781],[1.779650071540479],[0.02838873648259708],[95.85134362288377],[0.0012856420510214206],[0.5823168785015869],-3.009264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,12/2/2017,Stanford,https://stanforddaily.com/2017/12/02/brock-tur...,Stanford_12022017_1934.png,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076,11/20/2017,Stanford,https://stanforddaily.com/2017/11/20/qa-zerlin...,Stanford_11202017_1935.png,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077,11/14/2017,Stanford,https://stanforddaily.com/2017/11/14/joe-wrigh...,Stanford_11142017_1940.png,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1078,11/13/2017,Stanford,https://stanforddaily.com/2017/11/13/english-f...,Stanford_11132017_1941.png,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
